In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

2024-10-11 03:28:47.998035: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-11 03:28:48.009608: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-11 03:28:48.025620: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-11 03:28:48.030300: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-11 03:28:48.043176: I tensorflow/core/platform/cpu_feature_guar

In [3]:
holiday_events_df = pd.read_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/holidays_events.csv')
stores_df = pd.read_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/stores.csv')
transactions_df = pd.read_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/transactions.csv')
train_df = pd.read_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/train.csv')
test_df = pd.read_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/test.csv')
oil_df = pd.read_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/oil.csv')

In [4]:

holiday_events_df[holiday_events_df['transferred'] == True].head()
holiday_events_df.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [5]:
stores_df.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [6]:
transactions_df.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [7]:
train_df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [8]:
oil_df['dcoilwtico'] = oil_df['dcoilwtico'].bfill()
oil_df.head()

,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [9]:
full_train_df = train_df.merge(stores_df, how='left', on='store_nbr').merge(transactions_df, how='left', on=['date', 'store_nbr']).merge(oil_df, how='left', on='date').merge(holiday_events_df, how='left', on='date')
full_train_df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,transactions,dcoilwtico,type_y,locale,locale_name,description,transferred
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN,93.14,Holiday,National,Ecuador,Primer dia del ano,False
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN,93.14,Holiday,National,Ecuador,Primer dia del ano,False
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN,93.14,Holiday,National,Ecuador,Primer dia del ano,False
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN,93.14,Holiday,National,Ecuador,Primer dia del ano,False
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN,93.14,Holiday,National,Ecuador,Primer dia del ano,False


In [10]:
full_test_df = test_df.merge(stores_df, how='left', on='store_nbr').merge(transactions_df, how='left', on=['date', 'store_nbr']).merge(oil_df, how='left', on='date').merge(holiday_events_df, how='left', on='date')
full_test_df.head()

,id,date,store_nbr,family,onpromotion,city,state,type_x,cluster,transactions,dcoilwtico,type_y,locale,locale_name,description,transferred
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,NaN,46.8,NaN,NaN,NaN,NaN,NaN
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,NaN,46.8,NaN,NaN,NaN,NaN,NaN
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,NaN,46.8,NaN,NaN,NaN,NaN,NaN
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,NaN,46.8,NaN,NaN,NaN,NaN,NaN
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,NaN,46.8,NaN,NaN,NaN,NaN,NaN


In [10]:
full_train_df.dtypes

id                int64
date             object
store_nbr         int64
family           object
sales           float64
onpromotion       int64
city             object
state            object
type_x           object
cluster           int64
transactions    float64
dcoilwtico      float64
type_y           object
locale           object
locale_name      object
description      object
transferred      object
dtype: object

In [11]:
sample_df = pd.read_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/sample_submission.csv')
sample_df.count()

id       28512
sales    28512
dtype: int64

In [12]:
test_df = pd.read_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/test.csv')
test_df.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [13]:
full_train_df['proper_date']=pd.to_datetime(full_train_df['date'], format='%Y-%m-%d')
full_test_df['proper_date']=pd.to_datetime(full_test_df['date'], format='%Y-%m-%d')

full_train_df['day_of_week'] = full_train_df['proper_date'].dt.dayofweek
full_test_df['day_of_week'] = full_test_df['proper_date'].dt.dayofweek

full_train_df['month'] = full_train_df['proper_date'].dt.month
full_test_df['month'] = full_test_df['proper_date'].dt.month


# Encoding the type of products
family_label_encoder = LabelEncoder()
full_train_df['family_encoded'] = family_label_encoder.fit_transform(full_train_df['family'])
full_test_df['family_encoded'] = family_label_encoder.transform(full_test_df['family'])

#Encoding the citys and states
city_label_encoder = LabelEncoder()
full_train_df['city_encoded'] = city_label_encoder.fit_transform(full_train_df['city'])
full_test_df['city_encoded'] = city_label_encoder.transform(full_test_df['city'])

state_label_encoder = LabelEncoder()
full_train_df['state_encoded'] = state_label_encoder.fit_transform(full_train_df['state'])
full_test_df['state_encoded'] = state_label_encoder.transform(full_test_df['state'])

#Encoding the type of stores
type_store_label_encoder = LabelEncoder()
full_train_df['type_store_encoded'] = state_label_encoder.fit_transform(full_train_df['type_x'])
full_test_df['type_store_encoded'] = state_label_encoder.transform(full_test_df['type_x'])

#Transactions fillna
full_train_df['transactions'] = full_train_df['transactions'].fillna(0)
full_test_df['transactions'] = full_test_df['transactions'].fillna(0)
#Transactions Scale
transactions_scaler = MinMaxScaler()
full_train_df['transactions_scaled'] = transactions_scaler.fit_transform(full_train_df[['transactions']])
full_test_df['transactions_scaled'] = transactions_scaler.transform(full_test_df[['transactions']])

#Encoding the type of holidays
type_holiday_label_encoder = LabelEncoder()
full_train_df['type_y'].fillna('no_holiday', inplace=True)
full_test_df['type_y'].fillna('no_holiday', inplace=True)
full_train_df['type_holiday_encoded'] = state_label_encoder.fit_transform(full_train_df['type_y'])
full_test_df['type_holiday_encoded'] = state_label_encoder.transform(full_test_df['type_y'])

#Encoding the type of holidays
locale_label_encoder = LabelEncoder()
full_train_df['locale'].fillna('no_holiday', inplace=True)
full_test_df['locale'].fillna('no_holiday', inplace=True)
full_train_df['locale_encoded'] = state_label_encoder.fit_transform(full_train_df['locale'])
full_test_df['locale_encoded'] = state_label_encoder.transform(full_test_df['locale'])

#Encoding the type of holidays
transferred_label_encoder = LabelEncoder()
full_train_df['transferred'].fillna('no_holiday', inplace=True)
full_test_df['transferred'].fillna('no_holiday', inplace=True)
full_train_df['transferred'].replace({True: 'transferred', False: 'not_transferred'}, inplace=True)
full_test_df['transferred'].replace({True: 'transferred', False: 'not_transferred'}, inplace=True)
full_train_df['transferred_encoded'] = state_label_encoder.fit_transform(full_train_df['transferred'])
full_test_df['transferred_encoded'] = state_label_encoder.transform(full_test_df['transferred'])

oil_scaler = MinMaxScaler()
full_train_df['dcoilwtico'] = full_train_df['dcoilwtico'].bfill()
full_test_df['dcoilwtico'] = full_test_df['dcoilwtico'].bfill()
full_train_df['dcoilwtico_scaled'] = oil_scaler.fit_transform(full_train_df[['dcoilwtico']])
full_test_df['dcoilwtico_scaled'] = oil_scaler.transform(full_test_df[['dcoilwtico']])

sales_scaler = MinMaxScaler()
full_train_df['sales_scaled'] = sales_scaler.fit_transform(full_train_df[['sales']])
# full_test_df['sales_scaled'] = sales_scaler.transform(full_test_df[['sales']])

/tmp/ipykernel_59943/27171519.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  full_train_df['type_y'].fillna('no_holiday', inplace=True)
/tmp/ipykernel_59943/27171519.py:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tr

In [14]:
full_train_df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,...,family_encoded,city_encoded,state_encoded,type_store_encoded,transactions_scaled,type_holiday_encoded,locale_encoded,transferred_encoded,dcoilwtico_scaled,sales_scaled
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,...,0,18,12,3,0.0,3,1,1,0.792965,0.0
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,...,1,18,12,3,0.0,3,1,1,0.792965,0.0
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,...,2,18,12,3,0.0,3,1,1,0.792965,0.0
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,...,3,18,12,3,0.0,3,1,1,0.792965,0.0
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,...,4,18,12,3,0.0,3,1,1,0.792965,0.0


In [15]:
full_test_df.head()

,id,date,store_nbr,family,onpromotion,city,state,type_x,cluster,transactions,...,month,family_encoded,city_encoded,state_encoded,type_store_encoded,transactions_scaled,type_holiday_encoded,locale_encoded,transferred_encoded,dcoilwtico_scaled
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,0.0,...,8,0,18,12,3,0.0,6,3,0,0.244108
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,0.0,...,8,1,18,12,3,0.0,6,3,0,0.244108
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,0.0,...,8,2,18,12,3,0.0,6,3,0,0.244108
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,0.0,...,8,3,18,12,3,0.0,6,3,0,0.244108
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,0.0,...,8,4,18,12,3,0.0,6,3,0,0.244108


In [18]:
train_data=full_train_df[['store_nbr', 'onpromotion', 'cluster', 'day_of_week', 'month', 'family_encoded', 'city_encoded', 'state_encoded', 'type_store_encoded', 'type_holiday_encoded', 'locale_encoded', 'transferred_encoded', 'dcoilwtico_scaled']]
test_data=full_test_df[['store_nbr', 'onpromotion', 'cluster', 'day_of_week', 'month', 'family_encoded', 'city_encoded', 'state_encoded', 'type_store_encoded', 'type_holiday_encoded', 'locale_encoded', 'transferred_encoded', 'dcoilwtico_scaled']]

In [19]:
train_target=full_train_df['sales_scaled']

In [20]:
# Split the data into training and validation sets
train_data_split, val_data_split, train_target_split, val_target_split = train_test_split(train_data, train_target, test_size=0.2)

# Display the shapes of the resulting datasets
print(f"Training data shape: {train_data_split.shape}")
print(f"Validation data shape: {val_data_split.shape}")
print(f"Training target shape: {train_target_split.shape}")
print(f"Validation target shape: {val_target_split.shape}")

Training data shape: (2443478, 13)
Validation data shape: (610870, 13)
Training target shape: (2443478,)
Validation target shape: (610870,)


In [21]:
# Add a checkpoint to save the best model
checkpoint = tf.keras.callbacks.ModelCheckpoint('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/best_model.h5.keras', monitor='val_loss', save_best_only=True, mode='min')

# Define the model
model = Sequential()

# Add an LSTM layer with 50 units
model.add(LSTM(50, return_sequences=True, input_shape=(train_data_split.shape[1], 1)))
model.add(Dropout(0.2))

# Add another LSTM layer with 50 units
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))

# Add a Dense layer with 1 unit for the output
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

# Print the model summary
model.summary()

# Reshape the data to fit the model
train_data_reshaped = train_data_split.values.reshape((train_data_split.shape[0], train_data_split.shape[1], 1))
val_data_reshaped = val_data_split.values.reshape((val_data_split.shape[0], val_data_split.shape[1], 1))

# Fit the model
history = model.fit(train_data_reshaped, train_target_split, epochs=50, batch_size=256, validation_data=(val_data_reshaped, val_target_split), callbacks=[checkpoint])

I0000 00:00:1728628238.891809   59943 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728628238.930144   59943 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728628238.930637   59943 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728628238.938089   59943 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728628238.938597   59943 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 13, 50)         │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 13, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,651 (119.73 KB)

 Trainable params: 30,651 (119.73 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50


2024-10-11 03:30:41.737630: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8900


9545/9545 ━━━━━━━━━━━━━━━━━━━━ 125s 13ms/step - loss: 4.5381e-04 - mean_squared_error: 4.5381e-04 - val_loss: 5.8374e-05 - val_mean_squared_error: 5.8374e-05
Epoch 2/50
9545/9545 ━━━━━━━━━━━━━━━━━━━━ 129s 14ms/step - loss: 5.7814e-05 - mean_squared_error: 5.7814e-05 - val_loss: 4.8700e-05 - val_mean_squared_error: 4.8700e-05
Epoch 3/50
9545/9545 ━━━━━━━━━━━━━━━━━━━━ 136s 14ms/step - loss: 4.7693e-05 - mean_squared_error: 4.7693e-05 - val_loss: 3.4787e-05 - val_mean_squared_error: 3.4787e-05
Epoch 4/50
9545/9545 ━━━━━━━━━━━━━━━━━━━━ 131s 14ms/step - loss: 3.1031e-05 - mean_squared_error: 3.1031e-05 - val_loss: 2.2213e-05 - val_mean_squared_error: 2.2213e-05
Epoch 5/50
9545/9545 ━━━━━━━━━━━━━━━━━━━━ 141s 15ms/step - loss: 2.2036e-05 - mean_squared_error: 2.2036e-05 - val_loss: 1.7714e-05 - val_mean_squared_error: 1.7714e-05
Epoch 6/50
9545/9545 ━━━━━━━━━━━━━━━━━━━━ 137s 14ms/step - loss: 1.9847e-05 - mean_squared_error: 1.9847e-05 - val_loss: 1.5867e-05 - val_mean_squared_error: 1.5867e-

In [23]:
model.load_weights('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/best_model.h5.keras')

In [25]:
predictions = model.predict(test_data.values.reshape((test_data.shape[0], test_data.shape[1], 1)))
predictions_backscaled=sales_scaler.inverse_transform(predictions)
print(predictions_backscaled[:5])
submission_df=pd.DataFrame({'id': test_df['id'], 'sales': predictions_backscaled.flatten()})
submission_df.set_index('id', inplace = True)
submission_df.to_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/submission.csv')


891/891 ━━━━━━━━━━━━━━━━━━━━ 0s 364us/step
[[  25.980347]
 [  25.973957]
 [  38.185966]
 [2222.718   ]
 [  29.422771]]
